In [1]:
import pickle

with open('all_history.pickle', 'rb') as f:
    all_history = pickle.load(f)

In [4]:
# RPN, with one-hot connectives and variables.

from satml.expression import Type, _Expression, expr, pprint
from satml.solver     import simplify

import keras
from keras.preprocessing.sequence import pad_sequences

import numpy as np


_CONNECTIVE_ENCODING = {
    Type.AND: [1, 0],
    Type.OR:  [0, 1],
}
_STOP_WORD = -1


def rename_vars(an_exp):
    """
    Normalizes/renames all variables in a formula.
    Returns `renamed exp, mapping, num free variables`.
    """
    r_exp, mapping, current = _rename_vars(an_exp)
    return r_exp, mapping, (current - 1)


def _rename_vars(an_exp, mapping=dict(), current=1):
    if an_exp is None:
        return an_exp, mapping, current
    elif an_exp.typ == Type.VAR:
        val = an_exp.l_val

        if val in mapping:
            val = mapping[val]
        else:
            n_val = current
            mapping[val] = n_val
            val = n_val
            
            current += 1
        return expr((Type.VAR, val, None)), mapping, current
    elif an_exp.typ == Type.CONST:
        return an_exp, mapping, current
    
    l_exp, mapping, current = _rename_vars(an_exp.l_val, mapping, current)
    r_exp, mapping, current = _rename_vars(an_exp.r_val, mapping, current)
    
    return expr((an_exp.typ, l_exp, r_exp)), mapping, current


def _encode_expression(an_exp, num_variables):
    """
    Expects simplified, normalized formulas.
    See `satml.solver.simplify` and `rename_vars`.
    """
    if not isinstance(an_exp, _Expression):
        return an_exp
    
    if an_exp.typ == Type.VAR:
        var = an_exp.l_val
        return [to_categorical(var, num_variables * 2)]
    if an_exp.typ == Type.NOT:
        # Inner has to be a variable by the postconditions of `simplify`.
        var = -an_exp.l_val.l_val
        print(num_variables)
        return [to_categorical(var + num_variables, num_variables * 2)]
    
    return ([_CONNECTIVE_ENCODING[an_exp.typ]]
            + [[_STOP_WORD]]
            + _encode_expression(an_exp.l_val, num_variables)
            + [[_STOP_WORD]]
            + _encode_expression(an_exp.r_val, num_variables))


def encode_expression(an_exp):
    s_exp, mapping, num_vars = rename_vars(simplify(an_exp))
    print("Pre-encoding: {}\nEncoded: {}\nNum variables: {}".format(pprint(an_exp), pprint(s_exp), num_vars))
    
    return pad_sequences(
        _encode_expression(s_exp, num_vars),
        padding='post'
    ), mapping

In [5]:
from keras.utils import to_categorical

X, y = [], []

for expression, _, best_branch, _ in all_history:
    # We don't really care about expressions with <= 1 variables.
    if expression.typ != Type.AND and expression.typ != Type.OR:
        continue
        
    encoded, mapping = encode_expression(expression)    
    mapped_best_branch = mapping[best_branch]
    X.append(encoded)
    y.append(mapped_best_branch)

X = np.array(X).astype(np.int32)
y = to_categorical(np.array(y)).astype(np.int32)

Pre-encoding: ((((((((((((((((((((((6 ∨ (-3 ∨ -2)) ∧ (-6 ∨ 2)) ∧ -6) ∧ (3 ∨ -2)) ∧ (7 ∨ 3)) ∧ (7 ∨ -3)) ∧ (-6 ∨ -2)) ∧ (-7 ∨ 6)) ∧ -7) ∧ (6 ∨ -3)) ∧ (7 ∨ -3)) ∧ (7 ∨ 6)) ∧ (7 ∨ (6 ∨ -3))) ∧ (6 ∨ -3)) ∧ (7 ∨ (-6 ∨ 3))) ∧ 6) ∧ (7 ∨ -3)) ∧ (-6 ∨ -3)) ∧ -7) ∧ (6 ∨ 3)) ∧ (-6 ∨ -2)) ∧ 2)
Encoded: ((((((((((((((((((((((1 ∨ (-2 ∨ -3)) ∧ (-1 ∨ 3)) ∧ -1) ∧ (2 ∨ -3)) ∧ (4 ∨ 2)) ∧ (4 ∨ -2)) ∧ (-1 ∨ -3)) ∧ (-4 ∨ 1)) ∧ -4) ∧ (1 ∨ -2)) ∧ (4 ∨ -2)) ∧ (4 ∨ 1)) ∧ (4 ∨ (1 ∨ -2))) ∧ (1 ∨ -2)) ∧ (4 ∨ (-1 ∨ 2))) ∧ 1) ∧ (4 ∨ -2)) ∧ (-1 ∨ -2)) ∧ -4) ∧ (1 ∨ 2)) ∧ (-1 ∨ -3)) ∧ 3)
Num variables: 4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
Pre-encoding: ((((((((((((-5 ∨ -4) ∧ -4) ∧ (5 ∨ -4)) ∧ 4) ∧ -5) ∧ (5 ∨ 4)) ∧ -5) ∧ -5) ∧ -4) ∧ -5) ∧ (-5 ∨ -4)) ∧ (-5 ∨ 4))
Encoded: ((((((((((((-1 ∨ -2) ∧ -2) ∧ (1 ∨ -2)) ∧ 2) ∧ -1) ∧ (1 ∨ 2)) ∧ -1) ∧ -1) ∧ -2) ∧ -1) ∧ (-1 ∨ -2)) ∧ (-1 ∨ 2))
Num variables: 2
2
2
2
2
2
2
2
2
2
2
2
2
Pre-encoding: (((((((((((((((((((((((-5 ∨ (-3 ∨ 1)) ∧ (7 ∨ (5 ∨ -3))) ∧ (-7 ∨ (5 ∨ -1))) ∧ 3) ∧ 

IndexError: index 4 is out of bounds for axis 1 with size 2

In [ ]:
len(x[0])

print(x.dtype)
print(y.dtype)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(25))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, verbose=1, validation_data=(X_test, y_test))